In [1]:
import underworld as uw
import glucifer

mesh = uw.mesh._FeMesh_Regional(longExtent=120, latExtent=120, centroid=[1.0,2.0,0.0],elementRes=(10,10,5))

velocityField = uw.mesh.MeshVariable(mesh, nodeDofCount=3)
velocityField.data[mesh.specialSets["AllWalls_VertexSet"].data] = [0.0,1.0,0.0]

In [2]:
uw.libUnderworld.StgFEM._FeVariable_CreateNewEqnNumber( velocityField._cself )

gaussSwarm = uw.swarm.GaussIntegrationSwarm(mesh)

In [3]:
rot = uw.systems.sle.AssembledMatrix( velocityField, velocityField, rhs=None )
# rFn = mesh._boundaryNodeFn * mesh._getRadiusFn()
# nsFn = mesh._boundaryNodeFn * mesh._getNSFn()
# ewFn = mesh._boundaryNodeFn * mesh._getEWFn()


term = uw.systems.sle.MatrixAssemblyTerm_RotationDof(integrationSwarm=gaussSwarm, 
                                                     assembledObject = rot,
                                                     radialFn=mesh._rFn,
                                                     normalFn=mesh._normal,                                                
                                                     mesh=mesh)


uw.libUnderworld.StgFEM.StiffnessMatrix_Assemble( 
    rot._cself, 
    False,#self._stokesSLE._cself.removeBCs, 
    None, None );

In [4]:
vcopy = velocityField.copy()
uw.libUnderworld.StgFEM._FeVariable_CreateNewEqnNumber( vcopy._cself )


v1 = uw.systems.sle.SolutionVector(velocityField)
uw.libUnderworld.StgFEM.SolutionVector_LoadCurrentFeVariableValuesOntoVector( v1._cself );
ans = uw.systems.sle.SolutionVector(vcopy)

In [5]:
uw.libUnderworld.Underworld.AXequalsY( rot._cself, v1._cself, ans._cself )
vcopy.data[:]

array([[ 0.       ,  0.5      ,  0.8660254],
       [ 0.       ,  0.5      ,  0.8660254],
       [ 0.       ,  0.5      ,  0.8660254],
       ..., 
       [ 0.       ,  0.5      , -0.8660254],
       [ 0.       ,  0.5      , -0.8660254],
       [ 0.       ,  0.5      , -0.8660254]])

In [6]:
# create checkpoint function
def checkpoint( mesh, fieldDict, swarm, swarmDict, index,
                meshName='mesh', swarmName='swarm', 
                prefix=None, enable_xdmf=True):
    import os
    # Check the prefix is valid
    if prefix is not None:
        if not os.path.exists(prefix):
            raise ValueError("prefix given '{}' doesn't exist".format(prefix))
            
    if not isinstance(index, int):
        raise TypeError("'index' is not of type int")        
    ii = str(index)
    
    
    if mesh is not None:
        
        # Error check the mesh and fields
        if not isinstance(mesh, uw.mesh.FeMesh):
            raise TypeError("'mesh' is not of type uw.mesh.FeMesh")
        if not isinstance(fieldDict, dict):
            raise TypeError("'fieldDict' is not of type dict")
        for key, value in fieldDict.iteritems():
            if not isinstance( value, uw.mesh.MeshVariable ):
                raise TypeError("'fieldDict' must contain uw.mesh.MeshVariable elements")


        # see if we have already saved the mesh. It only needs to be saved once
        if not hasattr( checkpoint, 'mH' ):
            checkpoint.mH = mesh.save(prefix+meshName+".h5")
        mh = checkpoint.mH

        for key,value in fieldDict.iteritems():
            filename = prefix+key+'-'+ii
            handle = value.save(filename+'.h5')
            if enable_xdmf: value.xdmf(filename, handle, key, mh, meshName)
        
    # is there a swarm
    if swarm is not None:
        
        # Error check the swarms
        if not isinstance(swarm, uw.swarm.Swarm):
            raise TypeError("'swarm' is not of type uw.swarm.Swarm")
        if not isinstance(swarmDict, dict):
            raise TypeError("'swarmDict' is not of type dict")
        for key, value in swarmDict.iteritems():
            if not isinstance( value, uw.swarm.SwarmVariable ):
                raise TypeError("'fieldDict' must contain uw.swarm.SwarmVariable elements")
    
        sH = swarm.save(prefix+swarmName+"-"+ii+".h5")
        for key,value in swarmDict.iteritems():
            filename = prefix+key+'-'+ii
            handle = value.save(filename+'.h5')
            if enable_xdmf: value.xdmf(filename, handle, key, sH, swarmName)


In [7]:
fieldDict = {'velocity':velocityField, 'afterRot':vcopy}

checkpoint(mesh, fieldDict, None, None, index=0, prefix='./')
